In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

Média -> $\frac{1}{m} \sum_{i=1}^{m}$

Erro => $(h(x_i) - y_i)^2$

Erro quadrático médio -> $$ \sqrt{\frac{1}{m} \sum_{i=1}^{m}\left(h(x_i) - y_i\right)^{2}}$$


è muito comum a estratégia de:

- definir uma perda por amostra

- definir a perda total como a média das perdas individuais

**RMSE** (Root Mean Squared Error) é uma medida de erro absoluto que eleva os desvios ao quadrado para impedir que os desvios positivos e negativos se cancelem

# Nomenclatura

Dataset -> $D = {({(x_1, y_1),(x_2, y_2),(x_3, y_3),...,(x_m, y_m)})}$

$m$ -> Número de amostras

$n$ -> Número de features

Features -> $X = ({x_1,x_2,x_3,...,x_m})$

Target -> $Y = ({y_1,y_2,y_3,...,y_m})$

Previsões -> $Y^ = ({y_1,y_2,y_3,...,y_m})$

Iniciando a SEED da função random

In [2]:
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

Lendo o .csv

In [3]:
HOUSING_FILE = 'housing.csv'

def load_housing_data(housing_file=HOUSING_FILE):
    return pd.read_csv(housing_file)

housing = load_housing_data()

FileNotFoundError: [Errno 2] No such file or directory: 'housing.csv'

Separação entre dados de teste e de treinamento

In [ ]:
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(housing, test_size=0.2, random_state=RANDOM_SEED)

Código que insere uma nova coluna representando as faixas de renda, e, após isso, realizando uma separação estratificada para garantir representação proporcional das faixas de renda nos DataSets de treinamento e teste

In [ ]:
# Constroi uma coluna nova com categorias de renda fictícias.
housing['income_cat'] = np.ceil(housing['median_income'] / 1.5)
housing['income_cat'].where(housing['income_cat'] < 5, 5.0, inplace=True)


# Divide, de modo estratificado, o conjunto de dados.
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=RANDOM_SEED)
for train_index, test_index in split.split(housing, housing["income_cat"]):
    strat_train_set = housing.loc[train_index]
    strat_test_set = housing.loc[test_index]

In [ ]:
strat_train_set['income_cat'].value_counts() / len(strat_train_set)

income_cat
3.0    0.350594
2.0    0.318859
4.0    0.176296
5.0    0.114462
1.0    0.039789
Name: count, dtype: float64

In [ ]:
strat_test_set['income_cat'].value_counts() / len(strat_test_set)

income_cat
3.0    0.350533
2.0    0.318798
4.0    0.176357
5.0    0.114341
1.0    0.039971
Name: count, dtype: float64

In [ ]:
# Remove a coluna nova, que foi adicionada apenas temporariamente.
strat_train_set.drop(['income_cat'], axis=1, inplace=True)
strat_test_set.drop(['income_cat'], axis=1, inplace=True)

In [ ]:
strat_train_set.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16512 entries, 12655 to 19773
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           16512 non-null  float64
 1   latitude            16512 non-null  float64
 2   housing_median_age  16512 non-null  float64
 3   total_rooms         16512 non-null  float64
 4   total_bedrooms      16354 non-null  float64
 5   population          16512 non-null  float64
 6   households          16512 non-null  float64
 7   median_income       16512 non-null  float64
 8   median_house_value  16512 non-null  float64
 9   ocean_proximity     16512 non-null  object 
dtypes: float64(9), object(1)
memory usage: 1.4+ MB


In [ ]:
housing = strat_train_set.copy()

Criação da matriz de correlação, ajuda a ver o quanto uma variável implica na outra

In [ ]:
correlation_matrix = housing.drop(columns=['ocean_proximity']).corr()
correlation_matrix

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
longitude,1.000000,-0.924478,-0.105823,0.048909,0.076686,0.108071,0.063146,-0.019615,-0.047466
latitude,-0.924478,1.000000,0.005737,-0.039245,-0.072550,-0.115290,-0.077765,-0.075146,-0.142673
housing_median_age,-0.105823,0.005737,1.000000,-0.364535,-0.325101,-0.298737,-0.306473,-0.111315,0.114146
total_rooms,0.048909,-0.039245,-0.364535,1.000000,0.929391,0.855103,0.918396,0.200133,0.135140
total_bedrooms,0.076686,-0.072550,-0.325101,0.929391,1.000000,0.876324,0.980167,-0.009643,0.047781
population,0.108071,-0.115290,-0.298737,0.855103,0.876324,1.000000,0.904639,0.002421,-0.026882
households,0.063146,-0.077765,-0.306473,0.918396,0.980167,0.904639,1.000000,0.010869,0.064590
median_income,-0.019615,-0.075146,-0.111315,0.200133,-0.009643,0.002421,0.010869,1.000000,0.687151
median_house_value,-0.047466,-0.142673,0.114146,0.135140,0.047781,-0.026882,0.064590,0.687151,1.000000


Separando os dados entre variáveis indepedentes (as de entrada) e depedentes (de saída)

In [ ]:
# Variáveis independentes: dataset original menos a coluna de valores dependentes.
housing = strat_train_set.drop("median_house_value", axis=1)

# Variável dependente, também chamada de label.
housing_labels = strat_train_set["median_house_value"].copy()

In [ ]:
housing.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16512 entries, 12655 to 19773
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           16512 non-null  float64
 1   latitude            16512 non-null  float64
 2   housing_median_age  16512 non-null  float64
 3   total_rooms         16512 non-null  float64
 4   total_bedrooms      16354 non-null  float64
 5   population          16512 non-null  float64
 6   households          16512 non-null  float64
 7   median_income       16512 non-null  float64
 8   ocean_proximity     16512 non-null  object 
dtypes: float64(8), object(1)
memory usage: 1.3+ MB


Preenchendo as colunas vazias do DataFrame utilizando o SimpleImputer.

A estratégia utilizada é a "median", ou seja, o nosso imputer encontra as linhas vazias e coloca a mediana do valor no local.

### Transformadores no SKlearn

- `fit(x)` : aprende a transformações (mean,median,most common, constant)
- `transform(x)` : retorna o dataset transformado

In [ ]:
sample_incomplete_rows = housing[housing.isnull().any(axis=1)]
print(len(sample_incomplete_rows))

# Cria um imputer que substitui células inválidas (NaN) pela mediana dos valores da coluna à qual a célula pertence.
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='median')

# Antes de treinar o SimpleImputer, remover a coluna de dados categóricos. O dataset resultante tem apenas
# as variáveis independentes numéricas.
housing_num = housing.drop("ocean_proximity", axis=1)

# Agora treinar o Imputer. Isto vai causar o cálculo da mediana de cada coluna, 
# que ficará armazenado no Imputer para uso futuro. 
imputer.fit(housing_num)

# O Imputer agora tem as estatísticas desejadas armazenadas.
print("Estatísticas do Imputer:")
print(imputer.statistics_)

# Compare com as medianas do DataFrame:
print("Medianas")
print(housing_num.median().values)

# Aplicar o Imputer aos nossos dados. O valor de retorno é um ndarray do NumPy.
temp = imputer.transform(housing_num)
print(type(temp))

# Trabalhar com DataFrames geralmente é mais legal - dá para referenciar colunas por nome, ao invés de indices.
# Vamos transformar o ndarray em DataFrame.
housing_tr = pd.DataFrame(temp, columns=housing_num.columns)
print(type(housing_tr))

158
Estatísticas do Imputer:
[-118.51      34.26      29.      2119.       433.      1164.
  408.         3.54155]
Medianas
[-118.51      34.26      29.      2119.       433.      1164.
  408.         3.54155]
<class 'numpy.ndarray'>
<class 'pandas.core.frame.DataFrame'>


Verificação de colunas vazias no DataFrame Original

In [ ]:
housing[housing.isnull().any(axis=1)]

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity
1606,-122.08,37.88,26.0,2947.0,NaN,825.0,626.0,2.9330,NEAR BAY
10915,-117.87,33.73,45.0,2264.0,NaN,1970.0,499.0,3.4193,<1H OCEAN
19150,-122.70,38.35,14.0,2313.0,NaN,954.0,397.0,3.7813,<1H OCEAN
4186,-118.23,34.13,48.0,1308.0,NaN,835.0,294.0,4.2891,<1H OCEAN
16885,-122.40,37.58,26.0,3281.0,NaN,1145.0,480.0,6.3580,NEAR OCEAN
...,...,...,...,...,...,...,...,...,...
1350,-121.95,38.03,5.0,5526.0,NaN,3207.0,1012.0,4.0767,INLAND
4691,-118.37,34.07,50.0,2519.0,NaN,1117.0,516.0,4.3667,<1H OCEAN
9149,-118.50,34.46,17.0,10267.0,NaN,4956.0,1483.0,5.5061,<1H OCEAN
16757,-122.48,37.70,33.0,4492.0,NaN,3477.0,1537.0,3.0546,NEAR OCEAN


Verificação do DataFrame após o tratamento

In [ ]:
housing_tr[housing_tr.isnull().any(axis=1)].head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income


Realizando o encoding para variáveis categóricas do DataFrame.

O OrdinalEncoder transforma variáveis categóricas em números, e você poder ver o que cada número significa usando o `ordinal_encoder.categories_`

In [ ]:
from sklearn.preprocessing import OrdinalEncoder
ordinal_encoder = OrdinalEncoder()

housing_ocean_encoded = ordinal_encoder.fit_transform(housing[['ocean_proximity']])
housing_ocean_encoded[:10]

array([[1.],
       [4.],
       [1.],
       [4.],
       [0.],
       [3.],
       [0.],
       [0.],
       [0.],
       [0.]])

In [ ]:
ordinal_encoder.categories_

[array(['<1H OCEAN', 'INLAND', 'ISLAND', 'NEAR BAY', 'NEAR OCEAN'],
       dtype=object)]

Realizando o OneHotEncoding para as variáveis do DataFrame. Cria uma coluna específica para cada valor possível dentro da coluna original, onde booleanos são colocados para "se uma instância é ou não a variável daquela coluna", um vetor de features binárias.

O One-Hot Encoding recebe uma classe e retorna um vetor de variáveis booleanas, chamadas de variáveis dummy.

As colunas do nosso exemplo são ``'<1H OCEAN'``, ``'NEAR OCEAN'``, ``'INLAND'``, ``'NEAR BAY'`` e ``'ISLAND'``. Nesse caso, se o OneHotEncoder receber uma linha com ``'NEAR OCEAN'`` e retorna o seguinte vetor: $ [ \begin{array}{cc} 0 & 1 & 0 & 0 & 0\\\end{array} ] $

In [ ]:
# Cria o codificador.
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(categories='auto')

# Aprende a codificação e já aplica a mesma ao dataset fornecido. Todo transformador no sklearn
# tem os métodos fit() para aprender a transformação, e transform() para aplicá-la.
# O método fit_transform() faz os dois atos em sequência.
housing_cat_1hot = encoder.fit_transform(housing[['ocean_proximity']])

# O resultado da codificação é uma matriz esparsa em NumPy.
print(type(housing_cat_1hot))
print(housing_cat_1hot)

<class 'scipy.sparse._csr.csr_matrix'>
  (0, 1)	1.0
  (1, 4)	1.0
  (2, 1)	1.0
  (3, 4)	1.0
  (4, 0)	1.0
  (5, 3)	1.0
  (6, 0)	1.0
  (7, 0)	1.0
  (8, 0)	1.0
  (9, 0)	1.0
  (10, 1)	1.0
  (11, 0)	1.0
  (12, 1)	1.0
  (13, 1)	1.0
  (14, 4)	1.0
  (15, 0)	1.0
  (16, 0)	1.0
  (17, 0)	1.0
  (18, 3)	1.0
  (19, 0)	1.0
  (20, 1)	1.0
  (21, 3)	1.0
  (22, 1)	1.0
  (23, 0)	1.0
  (24, 1)	1.0
  :	:
  (16487, 1)	1.0
  (16488, 0)	1.0
  (16489, 4)	1.0
  (16490, 4)	1.0
  (16491, 1)	1.0
  (16492, 1)	1.0
  (16493, 0)	1.0
  (16494, 0)	1.0
  (16495, 0)	1.0
  (16496, 1)	1.0
  (16497, 0)	1.0
  (16498, 4)	1.0
  (16499, 0)	1.0
  (16500, 0)	1.0
  (16501, 1)	1.0
  (16502, 1)	1.0
  (16503, 1)	1.0
  (16504, 1)	1.0
  (16505, 0)	1.0
  (16506, 0)	1.0
  (16507, 0)	1.0
  (16508, 1)	1.0
  (16509, 0)	1.0
  (16510, 0)	1.0
  (16511, 1)	1.0


In [ ]:
# Convertendo em matriz densa só para observar melhor:
print(housing_cat_1hot.toarray()[:5])

# Você poderia também ter usado sparse=False na criação do OneHotEncoder.

arr = housing_cat_1hot.toarray()

[[0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0.]]


In [ ]:
encoder.categories_

[array(['<1H OCEAN', 'INLAND', 'ISLAND', 'NEAR BAY', 'NEAR OCEAN'],
       dtype=object)]

Utilizando o CombinedAttributesAdder para inicializar novas colunas no DataFrame, nesse caso cômodos por casas, população por número de casas e número de quartos por cômodos

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    # column index
    rooms_ix, bedrooms_ix, population_ix, household_ix = 3, 4, 5, 6
    
    def __init__(self, add_bedrooms_per_room=True):  # no *args or **kargs
        self.add_bedrooms_per_room = add_bedrooms_per_room
        
    def fit(self, X, y=None):
        return self  # nothing else to do

    def transform(self, X, y=None):

        # pega a coluna de rooms e a coluna de households e divide uma coluna pela outra, mas em np.array, gravando o resultado em outra variável
        rooms_per_household = \
            X[:, CombinedAttributesAdder.rooms_ix] / X[:, CombinedAttributesAdder.household_ix]
        
        # pega a coluna de popuação e a coluna de households e divide uma coluna pela outra, mas em np.array, gravando o resultado em outra variável
        population_per_household = \
            X[:, CombinedAttributesAdder.population_ix] / X[:, CombinedAttributesAdder.household_ix]


        if self.add_bedrooms_per_room:
            # pega a coluna de quartos (bedrooms) e a coluna de cômodos e divide uma coluna pela outra, mas em np.array, e grava o resultado em outra variável
            bedrooms_per_room = \
                X[:, CombinedAttributesAdder.bedrooms_ix] / X[:, CombinedAttributesAdder.rooms_ix]
            
            # retorna o array com as novas colunas adicionadas (concatenadas)
            return np.c_[X, rooms_per_household, population_per_household, bedrooms_per_room]
        else:
            # retorna o array com as novas colunas adicionadas (concatenadas)
            return np.c_[X, rooms_per_household, population_per_household]


attr_adder = CombinedAttributesAdder(add_bedrooms_per_room=False)

# montando o adder
housing_extra_attribs = attr_adder.transform(housing.values)

# Transformando em DataFrame, porque DataFrames são mais amigáveis.
columns_housing_extra_attribs = list(housing.columns) + ["rooms_per_household", "population_per_household"]
housing_extra_attribs = pd.DataFrame(housing_extra_attribs, columns=columns_housing_extra_attribs)
housing_extra_attribs.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity,rooms_per_household,population_per_household
0,-121.46,38.52,29.0,3873.0,797.0,2237.0,706.0,2.1736,INLAND,5.485836,3.168555
1,-117.23,33.09,7.0,5320.0,855.0,2015.0,768.0,6.3373,NEAR OCEAN,6.927083,2.623698
2,-119.04,35.37,44.0,1618.0,310.0,667.0,300.0,2.875,INLAND,5.393333,2.223333
3,-117.13,32.75,24.0,1877.0,519.0,898.0,483.0,2.2264,NEAR OCEAN,3.886128,1.859213
4,-118.7,34.28,27.0,3536.0,646.0,1837.0,580.0,4.4964,<1H OCEAN,6.096552,3.167241


Criação de uma Pipeline simples, onde preenchemos as lacunas do DataFrame utilizando a estratégia da mediana, depois fazemos o input das nova colunas desejadas e dpois utilizamos o StandarScaler para normalizar os dados (remover a média e dividir pelo desvio padrão).

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# Pipeline é uma cadeia de transformações, que rola um fit transform entre cada item da lista de transformações, alterando o DF original entre as etapas (o resultado de uma sendo a entrada da outra)
num_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="median")),
        ('attribs_adder', CombinedAttributesAdder()),

        # o StandardScaler remove a média de cada coluna e a divide pelo desvio padrão
        ('std_scaler', StandardScaler()),
    ])

housing_num_tr = num_pipeline.fit_transform(housing_num)
housing_num_tr

array([[-0.94135046,  1.34743822,  0.02756357, ...,  0.01739526,
         0.00622264, -0.12112176],
       [ 1.17178212, -1.19243966, -1.72201763, ...,  0.56925554,
        -0.04081077, -0.81086696],
       [ 0.26758118, -0.1259716 ,  1.22045984, ..., -0.01802432,
        -0.07537122, -0.33827252],
       ...,
       [-1.5707942 ,  1.31001828,  1.53856552, ..., -0.5092404 ,
        -0.03743619,  0.32286937],
       [-1.56080303,  1.2492109 , -1.1653327 , ...,  0.32814891,
        -0.05915604, -0.45702273],
       [-1.28105026,  2.02567448, -0.13148926, ...,  0.01407228,
         0.00657083, -0.12169672]])

Pipelina apenas utilizada para a Realização do OneHotEncoding na coluna de variáveis categóricas do DataFrame

In [ ]:
cat_pipeline = Pipeline([
        ('cat_encoder', OneHotEncoder(sparse=False)),
    ])

housing_cat_tr = cat_pipeline.fit_transform(housing[['ocean_proximity']])
housing_cat_tr

c:\Users\jalfr\Desktop\Machine Learning\aula_02_03\env\lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


array([[0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 1., 0., 0., 0.],
       ...,
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.]])

O ColumnTransformer é como se fosse uma pipeline, recebe uma lista de operações com os tranformadores que você vai querer aplicar, mas por colunas.

Você escolhe quais colunas que passam por cada transformador, e as não especificadas passam reto pelo processo

In [ ]:
from sklearn.compose import ColumnTransformer

num_attribs = list(housing_num)
cat_attribs = ["ocean_proximity"]

full_pipeline = ColumnTransformer([
        ("num", num_pipeline, num_attribs),
        ("cat", OneHotEncoder(sparse=False), cat_attribs),
    ])

housing_prepared = full_pipeline.fit_transform(housing)

c:\Users\jalfr\Desktop\Machine Learning\aula_02_03\env\lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Abaixo, temos o resultado final da pipeline no DataFrame antigo.

In [ ]:
housing_prepared = full_pipeline.fit_transform(housing)
housing_prepared[:5]

c:\Users\jalfr\Desktop\Machine Learning\aula_02_03\env\lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


array([[-0.94135046,  1.34743822,  0.02756357,  0.58477745,  0.64037127,
         0.73260236,  0.55628602, -0.8936472 ,  0.01739526,  0.00622264,
        -0.12112176,  0.        ,  1.        ,  0.        ,  0.        ,
         0.        ],
       [ 1.17178212, -1.19243966, -1.72201763,  1.26146668,  0.78156132,
         0.53361152,  0.72131799,  1.292168  ,  0.56925554, -0.04081077,
        -0.81086696,  0.        ,  0.        ,  0.        ,  0.        ,
         1.        ],
       [ 0.26758118, -0.1259716 ,  1.22045984, -0.46977281, -0.54513828,
        -0.67467519, -0.52440722, -0.52543365, -0.01802432, -0.07537122,
        -0.33827252,  0.        ,  1.        ,  0.        ,  0.        ,
         0.        ],
       [ 1.22173797, -1.35147437, -0.37006852, -0.34865152, -0.03636724,
        -0.46761716, -0.03729672, -0.86592882, -0.59513997, -0.10680295,
         0.96120521,  0.        ,  0.        ,  0.        ,  0.        ,
         1.        ],
       [ 0.43743108, -0.63581817, -0

Utilizando o modelo de regressão linear com as variaváveis independentes e depedentes

Modelos Preditivos do SciKitLearn tem apenas duas funções:

- `fit(x,y)`: faz o treinamento do modelo
- `predict(x)`: retorna a previsão de $y_{est}$

In [ ]:
# Variável dependente, também chamada de label.
housing_labels = strat_train_set["median_house_value"].copy()

In [ ]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()

# fit é o que treina o modelo
lin_reg.fit(housing_prepared, housing_labels)

LinearRegression()

Um teste com o Regressor Linear

In [ ]:
# Seleciona 5 pontos do conjunto de treinamento.
some_data = housing.iloc[:5]
some_labels = housing_labels.iloc[:5]

# Prepara os dados - não se esqueça deste passo.
some_data_prepared = full_pipeline.transform(some_data)

# Para obter as previsões, basta chamar o método predict()
predicted_labels = lin_reg.predict(some_data_prepared)
print("Predição: {}".format(predicted_labels))

# Compare com os valores originais:
print("Original: {}".format(some_labels.values))

Predição: [ 85657.90192014 305492.60737488 152056.46122456 186095.70946094
 244550.67966089]
Original: [ 72100. 279600.  82700. 112500. 238300.]


Treinando o modelo preditivo e medindo o erro quadrático médio do seu resultado

In [ ]:
from sklearn.metrics import mean_squared_error

housing_predictions = lin_reg.predict(housing_prepared)
lin_mse = mean_squared_error(housing_labels, housing_predictions)
lin_rmse = np.sqrt(lin_mse)
print('Regressão linear: RMSE = {:.2f}'.format(lin_rmse))

Regressão linear: RMSE = 68627.87


Treinando o modelo preditivo de classificação por árvore de decisão

In [ ]:
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor(random_state=RANDOM_SEED)
tree_reg.fit(housing_prepared, housing_labels)

DecisionTreeRegressor(random_state=42)

In [ ]:
predicted_labels = tree_reg.predict(some_data_prepared)
print("Predição: {}".format(predicted_labels))
print("Original: {}".format(some_labels.values))

Predição: [ 72100. 279600.  82700. 112500. 238300.]
Original: [ 72100. 279600.  82700. 112500. 238300.]


Testando o erro quadrático médio do algoritmo

In [ ]:
housing_predictions = tree_reg.predict(housing_prepared)
tree_mse = mean_squared_error(housing_labels, housing_predictions)
tree_rmse = np.sqrt(tree_mse)
print('Regressão linear: RMSE = {:.2f}'.format(tree_rmse))

Regressão linear: RMSE = 0.00


Erro é 0, portanto, ocorreu o processo de OverFitting

Realização de um Train/Test split: treinamos com um set de dados, e testamos a sua acuráia com outro. Neste caso, estamos criando um X de validação e um Y de validação, para que nossos modelos estejam de boa

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(housing_prepared, housing_labels, test_size=0.2, random_state=RANDOM_SEED)

Regressão Linear

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

lin_reg = LinearRegression()

lin_reg.fit(X_train, y_train)

y_pred = lin_reg.predict(X_valid)
lin_rmse = np.sqrt(mean_squared_error(y_valid, y_pred))
print('Regressão Linear: RMSE = {:.2f}'.format(lin_rmse))

Regressão Linear: RMSE = 67700.15


Árvore de Decisão

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error

tree_reg = DecisionTreeRegressor(random_state=RANDOM_SEED)

tree_reg.fit(X_train, y_train)

y_pred = tree_reg.predict(X_valid)
tree_rmse = np.sqrt(mean_squared_error(y_valid, y_pred))
print('Regressão por Ávore de Decisão: RMSE = {:.2f}'.format(tree_rmse))

Regressão por Ávore de Decisão: RMSE = 72649.69


Regressão de Random Forest (várias árvores de classificação)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

forest_reg = RandomForestRegressor(n_estimators=10, random_state=RANDOM_SEED)

forest_reg.fit(X_train, y_train)

y_pred = forest_reg.predict(X_valid)
forest_rmse = np.sqrt(mean_squared_error(y_valid, y_pred))
print('Regressão random forest: RMSE = {:.2f}'.format(forest_rmse))

Regressão random forest: RMSE = 52413.54


Parece que o random forest é melhor que os outros!

Mas talvez todos esses resultados sejam pura sorte: como saber? Podemos repetir esses experimentos com partições diferentes e ver se o resultado se mantém. O scikit-learn já tem ferramentas para ajudar nessa tarefa:

O código abaixo executa ***n-fold cross validation*** (neste caso, $n=10$). A função ``cross_val_score`` divide o conjunto de treinamento em $n$ partes e executa o procedimento de testes (treinar modelo, prever, medir erro) $n$ vezes - uma para cada partição. A cada ensaio a partição da vez é separada como conjunto de teste, e as demais compõe o conjunto de treinamento.

Uma vantagem desta abordagem é que agora podemos ver a faixa de desempenhos do modelo.

Validação Cruzada: quando fazemos a separação Treino/Validação/Teste, dividimos o nosso DataSet de treinamento e dividimos ele em dois pedaços ($x_{treino}$ e $x_{validacão}$). Na validação Cruzada, pegamos a metade de validação e testes e dividimos em um número ainda menor de pedaços, e realizamos treinamento e validação nesses pedaços da seguinte maneira: escolher um pedaço de teste (final) e treinar o modelo com os pedaços restantes.

Modelo de Validação Cruzada aplicada a regressão linear

In [ ]:
from sklearn.model_selection import cross_val_score

lin_scores = cross_val_score(lin_reg, housing_prepared, housing_labels, scoring="neg_mean_squared_error", cv=10)
lin_rmse_scores = np.sqrt(-lin_scores)


def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())

display_scores(lin_rmse_scores)

Scores: [71762.76364394 64114.99166359 67771.17124356 68635.19072082
 66846.14089488 72528.03725385 73997.08050233 68802.33629334
 66443.28836884 70139.79923956]
Mean: 69104.07998247063
Standard deviation: 2880.328209818065


Aplicado ao Random Tree

In [ ]:
tree_scores = cross_val_score(tree_reg, housing_prepared, housing_labels, scoring="neg_mean_squared_error", cv=10)
tree_rmse_scores = np.sqrt(-tree_scores)
display_scores(tree_rmse_scores)

Scores: [72831.45749112 69973.18438322 69528.56551415 72517.78229792
 69145.50006909 79094.74123727 68960.045444   73344.50225684
 69826.02473916 71077.09753998]
Mean: 71629.89009727491
Standard deviation: 2914.035468468928


Comparação de TTest

In [ ]:
from scipy.stats import ttest_ind

# alpha é geralmento 0.05
# para afirmar que a hipótese nula está errada, tem que avaliar o pvalue e ver se ele é menor que o valor de alpha escolhido

# In statistical hypothesis testing, the p-value serves as an alternative to rejection points to provide the smallest level of significance at which the null hypothesis would be rejected. 
# The p-value is the probability of obtaining results at least as extreme as the observed results of a statistical hypothesis test, assuming that the null hypothesis is correct. 
# A smaller p-value means that there is stronger evidence in favor of the alternative hypothesis.

# If the p-value is less than or equal to the significance level, reject the null hypothesis in favor of the alternative hypothesis.
# However, the risk of rejecting the null hypothesis is often higher than the p-value, especially when looking at a single study or when using small sample sizes.
# The choice of significance level at which you reject H0 is arbitrary, with conventionally the 5%, 1%, and 0.1% levels used.1


ttest_ind(lin_rmse_scores,tree_rmse_scores,equal_var=False)

TtestResult(statistic=-1.8493701883396976, pvalue=0.08089762185936315, df=17.997563986491905)

Aplicado ao Random Forest

In [ ]:
forest_scores = cross_val_score(forest_reg, housing_prepared, housing_labels, scoring="neg_mean_squared_error", cv=10, n_jobs=-1)
forest_rmse_scores = np.sqrt(-forest_scores)
display_scores(forest_rmse_scores)

Scores: [53519.05518628 50467.33817051 48924.16513902 53771.72056856
 50810.90996358 54876.09682033 56012.79985518 52256.88927227
 51527.73185039 55762.56008531]
Mean: 52792.92669114079
Standard deviation: 2262.8151900582


In [ ]:
from scipy.stats import ttest_ind

# alpha é geralmento 0.05
# para afirmar que a hipótese nula está errada, tem que avaliar o pvalue e ver se ele é menor que o valor de alpha escolhido

# In statistical hypothesis testing, the p-value serves as an alternative to rejection points to provide the smallest level of significance at which the null hypothesis would be rejected. 
# The p-value is the probability of obtaining results at least as extreme as the observed results of a statistical hypothesis test, assuming that the null hypothesis is correct. 
# A smaller p-value means that there is stronger evidence in favor of the alternative hypothesis.

# If the p-value is less than or equal to the significance level, reject the null hypothesis in favor of the alternative hypothesis.
# However, the risk of rejecting the null hypothesis is often higher than the p-value, especially when looking at a single study or when using small sample sizes.
# The choice of significance level at which you reject H0 is arbitrary, with conventionally the 5%, 1%, and 0.1% levels used.1


ttest_ind(lin_rmse_scores,forest_rmse_scores,equal_var=False)

TtestResult(statistic=13.359318769367425, pvalue=1.8498893257958064e-10, df=17.044881939960536)

Mas afinal, o que é um modelo de regressão? É uma função que transforma os dados de entrada em um valor de saída, e que também pode depender de alguns **parâmetros**:

$$y = h(x; \theta)$$

Treinar o modelo é ajustar os parâmetros do modelo para maximizar o desempenho preditivo deste. Para tanto devemos usar um algoritmo de treinamento. Cada classe de modelos demanda seu próprio algoritmo de treinamento, vamos estudar isso em detalhes mais tarde.

$$\theta_{opt} = \text{argmin}_{\theta} \{ \text{RMSE}\left(X_{\text{train}}, y_{\text{train}}, h_{\theta} \right) \}$$

Plot twist: os algoritmos de treinamento em si *também* tem seus parâmetros! Ademais, os modelos tem parâmetros que especificam sub-classes de modelos, e diferem dos parâmetros voltados ao "ajuste fino". A esses meta-parâmetros chamamos **hiperparâmetros**.

Os parâmetros regulares são ajustados pelo método ``fit()`` dos regressores. Como ajustar os hiperparâmetros? A abordagem mais simples é testar vários valores e ver o que funciona! Existem abordagens mais sofisticadas, que discutiremos depois, mas por hoje vamos testar uma dessas abordagens "força-bruta" chamada *grid search*.

Funciona assim: escolha alguns valores possíveis de hiperparâmetros, e teste todas as combinações. Vamos aplicar isso ao regressor random forest. Não se preocupe com o significado destes hiperparâmetros por enquanto, vamos estudar isso em detalhes depois.

Em scikit-learn, temos uma classe ``GridSearchCV`` para fazer isso. *AVISO*: vai demorar!

In [ ]:
from sklearn.model_selection import GridSearchCV
from timeit import default_timer

# lista de experimentos (nome do hiperparâmetro e opções que quer testar)
param_grid = [
    # try 6 (2×3) combinations of hyperparameters
    {'n_estimators': [10, 30], 'max_features': [4, 6, 8]},
    # then try 4 (1x2×2) combinations with bootstrap set as False
    {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [3, 4]},
  ]

forest_reg = RandomForestRegressor(random_state=RANDOM_SEED)

# train across 5 folds, that's a total of (6+4)*5=50 rounds of training 
grid_search = GridSearchCV(forest_reg, param_grid, cv=5, scoring='neg_mean_squared_error', return_train_score=True, n_jobs=-1)

t1 = default_timer()
grid_search.fit(housing_prepared, housing_labels)
t2 = default_timer()

print(f'Tempo gasto: {t2 - t1:.2f} s')

Tempo gasto: 39.16 s


In [ ]:
grid_search.best_params_

{'max_features': 8, 'n_estimators': 30}

O ``GridSearch`` já retorna o melhor modelo treinado:

In [ ]:
grid_search.best_estimator_

RandomForestRegressor(max_features=8, n_estimators=30, random_state=42)

Para alguns modelos de machine learning podemos obter a importância relativa das características no processo de predição. Esta informação é importante para entender melhor nosso problema. De fato, um dos usos bastante importantes do machine learning é exatamente isso: usar o machine learning para entender melhor o problema em si!

In [ ]:
feature_importances = grid_search.best_estimator_.feature_importances_
feature_importances

array([6.96542523e-02, 6.04213840e-02, 4.21882202e-02, 1.52450557e-02,
       1.55545295e-02, 1.58491147e-02, 1.49346552e-02, 3.79009225e-01,
       5.47789150e-02, 1.07031322e-01, 4.82031213e-02, 6.79266007e-03,
       1.65706303e-01, 7.83480660e-05, 1.52473276e-03, 3.02816106e-03])

In [ ]:
extra_attribs = ["rooms_per_hhold", "pop_per_hhold", "bedrooms_per_room"]
cat_encoder = cat_pipeline.named_steps["cat_encoder"]
cat_one_hot_attribs = list(cat_encoder.categories_[0])
attributes = num_attribs + extra_attribs + cat_one_hot_attribs
sorted(zip(feature_importances, attributes), reverse=True)

[(0.3790092248170967, 'median_income'),
 (0.16570630316895876, 'INLAND'),
 (0.10703132208204355, 'pop_per_hhold'),
 (0.06965425227942929, 'longitude'),
 (0.0604213840080722, 'latitude'),
 (0.054778915018283726, 'rooms_per_hhold'),
 (0.048203121338269206, 'bedrooms_per_room'),
 (0.04218822024391753, 'housing_median_age'),
 (0.015849114744428634, 'population'),
 (0.015554529490469328, 'total_bedrooms'),
 (0.01524505568840977, 'total_rooms'),
 (0.014934655161887772, 'households'),
 (0.006792660074259966, '<1H OCEAN'),
 (0.0030281610628962747, 'NEAR OCEAN'),
 (0.0015247327555504937, 'NEAR BAY'),
 (7.834806602687504e-05, 'ISLAND')]

- Usamos validação cruzada para achar a melhor família de regressores para nosso modelo. Note que nesta etapa não ajustamos hiperparâmetros, apenas confiamos nos valores default.

- Usamos novamente validação cruzada para achar os melhores hiperparâmetros, com busca no espaço de hiperparâmetros.

Agora temos o nosso melhor modelo, treinado na forja da validação cruzada! Chegou finalmente a hora de medir o desempenho do regressor no conjunto de testes!

In [ ]:
final_model = grid_search.best_estimator_

X_test = strat_test_set.drop("median_house_value", axis=1)
y_test = strat_test_set["median_house_value"].copy()

X_test_prepared = full_pipeline.transform(X_test)
final_predictions = final_model.predict(X_test_prepared)

final_mse = mean_squared_error(y_test, final_predictions)
final_rmse = np.sqrt(final_mse)

print("RMSE = {}".format(final_rmse))

RMSE = 47873.26095812988
